In [1]:
import os
import pandas as pd
from network_wrangler import RoadwayNetwork
from network_wrangler import TransitNetwork
from network_wrangler import ProjectCard
from network_wrangler import Scenario
from network_wrangler import WranglerLogger
from lasso import ModelRoadwayNetwork
import warnings

warnings.filterwarnings("ignore")
import yaml
import sys
import ast

## Read a Config File

Stored as `my_config` variable

In [11]:
MY_CONFIG_FILE = os.path.join(
    os.path.dirname(os.path.abspath('')), "examples", "settings","my_config.yaml"
)

with open(MY_CONFIG_FILE) as f:
        my_config = yaml.safe_load(f)
        
import json
print(json.dumps(my_config, indent=2))

{
  "base_scenario": {
    "input_dir": "examples/stpaul",
    "shape_file_name": "shape.geojson",
    "link_file_name": "link.json",
    "node_file_name": "node.geojson",
    "transit_directory": "",
    "validate_network": false
  },
  "scenario": {
    "project_cards_filenames": [],
    "card_directory": "../../network_wrangler/examples/stpaul/project_cards",
    "glob_search": "*attribute*.yml",
    "tags": []
  },
  "output": {
    "format": "geojson",
    "directory": "",
    "prefix": ""
  }
}


## Create a Base Scenario

In [15]:
base_project_path = os.path.dirname(os.path.abspath(''))

base_scenario = Scenario.create_base_scenario(
        my_config["base_scenario"]["shape_file_name"],
        my_config["base_scenario"]["link_file_name"],
        my_config["base_scenario"]["node_file_name"],
        base_dir=os.path.join(base_project_path,my_config["base_scenario"]["input_dir"]),
    )

SystemExit: RoadwayNetwork: Data doesn't conform to schema

#### Create project cards from projects that are explicitely specified in config


In [22]:
if len(my_config["scenario"]["project_cards_filenames"]) > 0:
    project_cards_list = [
        ProjectCard.read(filename, validate=False)
        for filename in my_config["scenario"]["project_cards_filenames"]
    ]
    project_cards_list

## Create Scenario

In [ ]:
my_scenario = Scenario.create_scenario(
    base_scenario=base_scenario,
    card_directory=my_config["scenario"]["card_directory"],
    tags=my_config["scenario"]["tags"],
    project_cards_list=project_cards_list,
    glob_search=my_config["scenario"]["glob_search"],
)

### Apply all projects in scenario

In [ ]:
my_scenario.apply_all_projects()

# Write out as MetCouncil Model Network

model_road_net = ModelRoadwayNetwork.from_RoadwayNetwork(
    my_scenario.road_net, parameters=my_config.get("my_parameters", {})
)

In [ ]:
model_road_net.write_roadway_as_shp()